In [1]:
from DataPreprocessing.python.Data import *

import pandas as pd
import numpy as np


## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

# This is new, it will allow us to interact with
# 3d plots in the notebook
from mpl_toolkits.mplot3d import Axes3D


## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

In [19]:
rootData = Data("/Users/czkaiweb/Research/ErdosBootCamp/Project/ProjectData/Root_Insurance_data.csv")
rootData.loadData()
rootData.factorizeData()
rootData.splitData(fraction=[0.5,0.3,0.2],random_seed=42)
data_train = rootData.getTrainDataCopy()
data_validation = rootData.getValidationDataCopy()
data_test  = rootData.getTestDataCopy()

In [20]:
X_train = data_train.iloc[:, 0:4].values
y_train = data_train.iloc[:, -1].values

X_validation = data_validation.iloc[:, 0:4].values
y_validation = data_validation.iloc[:, -1].values

X_test = data_test.iloc[:, 0:4].values
y_test = data_test.iloc[:, -1].values

In [21]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_validation = sc.transform(X_validation)
X_test = sc.transform(X_test)

In [22]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5,random_state=42).fit(X_train, y_train)

y_vpred = clf.predict(X_validation)

In [23]:
clf.score(X_validation, y_validation)

0.924

In [28]:
y_train.mean()

0.0762